# Adding Definitions to JSON File
This notebook is required to load the current JSON file of the glossary and to enrich it with additional information. In our case the definition texts for each glossary term. 

## 0. Requirements

In [560]:
# import required libraries
import json
import requests
import pandas as pd

## 1. Get JSON File 

### 1.1 Retrieve from Website 
The following code retrieves the current JSON file from the website http://www.klimadiskurs.info/ to be able to add the definition texts.

In [561]:
# retrieve JSON file and save input to "glossary" variable 
response = requests.get("http://www.klimadiskurs.info/download/json")
glossary = json.loads(response.text)

### 1.2 Retrieve from local file 
Alternatively, the file can be opened from the local disk as follows: 

In [562]:
with open('../files/glossary.json') as f:
    glossary = json.load(f)

This is how a sample entry in the JSON file looks like:

In [563]:
glossary["Klimaabzockerei"]#["definition"]

{'term': 'Klimaabzockerei',
 'id': 1,
 'definition': '',
 'sources': [],
 'related': [],
 'spellings': ['Klimaabzockerei', 'Klima-Abzockerei'],
 'examples': ['Ganz klar, dass einige Wirtschaftszweige und einige Gruppierungen mit der Klima-Abzockerei viel Geld verdienen'],
 'association': [0]}

## 2. Load Knowledge Base with Definitions
To load the knowledge base containing the final definition texts please use the following code.

In [564]:
knowledge_base = pd.read_csv("../output/knowledge_base_updated.csv")

## 3. Add Definitions to JSON
Next, we will use the function `add_definitions` to add the definition strings to the JSON file.

In [565]:
def add_definitions(json):
    
    # for each glossary term in json file 
    for term in json:

        # retrieve index of term in data frame 
        idx = knowledge_base.index[knowledge_base['original'] == term.lower()][0]

        # retrieve definition from knowledge base 
        definition = knowledge_base['full_definition'].iloc[idx]

        # add final definitions from knowledge base to json file 
        json[term]["definition"] = definition

    return json

In [566]:
# apply function and save to variable 
glossary_json = add_definitions(glossary)

## 4. Save JSON File
Then we will save the new json file to `glossary_updated.json` in the directory `files`.

In [567]:
with open("../files/glossary_updated.json", "w+") as f:
    json.dump(glossary_json, f)

## Clean Context

In [529]:
pro_context = pd.read_csv("../../R/output/pro_context.csv")
con_context = pd.read_csv("../../R/output/con_context.csv")

#pro_context['index'] = pro_context.index
#pro_context = pro_context.reset_index()
#con_context = con_context.reset_index()

In [530]:
def clean_context(df):
   
    """
    This function checks for each compound word whether it is indeed mentioned in its full form in the key word phrase.
    Rows that do not contain the exact compound word in its key word phrase are discarded from the data frame. 
    Arg: 
        df: a data frame containing the compound and its key word phrases.
    Returns: 
        Nothing.
    """ 

    # initiate list of rows that we have to drop
    to_drop = []
    
    # for each row in context data frame 
    for idx, row in df.iterrows():

        # retrieve compound word for according row 
        compound = row.pattern
        # retrieve compound forms for the compound word
        compound_forms = knowledge_base[knowledge_base.original == compound].compound_forms.tolist()
        # convert nested list into simplified list of compound forms 
        compound_forms = [item for sublist in compound_forms for item in sublist]

        print(compound_forms)
        # if the keyword phrase does not contain any of the compound forms 
        if not df.iloc[idx].str.contains(" |".join(compound_forms)+" ", case=False).keyword:
            
            # append this row index to the "to drop" list
            to_drop.append(idx)

        # if keyword phrase contains one of the compound forms
        else:
            # do nothing
            pass

    # drop rows that are contained in to_drop list
    df.drop(to_drop, inplace=True)
    
    # reset index of data frame 
    df.reset_index(drop=True, inplace=True)

In [531]:
# apply function to both context data frames 
clean_context(pro_context)
clean_context(con_context)

['klimaaktivismus']
['klimaaktivismus']
['klimaaktivismus']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivisten', 'klimaaktivist']
['klimaaktivist

In [532]:
len(pro_context)

541

In [533]:
len(con_context)

1456

In [534]:
for element in con_context[con_context.pattern == "klimafreund"].keyword:
    print(element)
    print("_"*10)

Moderne Heuschrecken sind geschworene „ Klimafreunde " Die für Eisbären begeisterten Schüler scheinen sich gar nicht dafür zu interessieren , dass die allerübelsten Heuschrecken heute in der Wall Street oder den Bankenvierteln anderer Finanzmetropolen sitzen und sich diebisch freuen , dass Kinder weltweit für ihre Profite trommeln .
__________


## SARCASM

In [540]:
def find_quotations(df):
    
    """
    This function checks for each key word sentence whether the compound word is mentioned in quotation marks. 
    Arg: 
        df: a data frame containing the compound and a key word phrase.
    Returns: 
        A new column with the binary label 1 if the compound is used in quotation marks, else 0.
    """ 
    
    # for each row in the data frame
    for idx, row in df.iterrows():
        
        string = row['keyword'].lower() # retrieve key word phrase
        compound = row['pattern'] # retrieve compound word
       
        # retrieve exact positions of compound word in the string
        for match in re.finditer(compound, string):
            start = match.start() # get start position
            end = match.end() # and end position
            
            try: 
                # if 1 position before the starting point, we have one of the following quotation marks
                if string[start-1] == '"' or string[start-1] == """'""" or string[start-1] == """„""":
                    x = True # set x to True
                    
                # or if 2 position before the starting point, we have one of the following quotation marks
                elif string[start-2] == '"' or str1[start-2] == """'""" or string[start-2] == """„""":
                    x = True # set x to True
                                        
                # else, x is False
                else:
                    x = False
                    
            except:
                x = False
                y = False

            try:
                # if 1 position after the end point, we have one of the following quotation marks
                if string[end+1] == '"' or string[end+1] == """'""" or string[end+1] == """“""":
                    y = True # set y to True
                                        
                # if 2 positions after the end point, we have one of the following quotation marks
                elif string[end+2] == '"' or string[end+2] == """'""" or string[end+2] == """“""":
                    y = True # set y to True
                                        
                # else, y is False
                else:
                    y = False 
                    
            except:
                x = False
                y = False

            # if we have 2 quotations marks surrounding the compound
            if x and y == True:
                df.at[idx,'quotes'] = 1 # set value in new column to 1 
 
            # if not:
            else:
                df.at[idx,'quotes'] = 0 # set value in new column to 0

In [541]:
find_quotations(pro_context)
find_quotations(con_context)

In [542]:
con_context[con_context.pattern == "klimafreund"].quotes.values

array([1.])

In [525]:
pro_context.quotes = pro_context.quotes.astype('int32')
con_context.quotes = con_context.quotes.astype('int32')

In [538]:
con_context[con_context.pattern == "klimafreund"].quotes.values

array([1.])

In [496]:
con_sarcasm = con_context.groupby('pattern')['quotes'].mean().to_dict()

In [539]:
con_sarcasm["klimafreund"]

1.0

In [500]:
pro_context.quotes = pro_context.quotes.astype('int32')
con_context.quotes = con_context.quotes.astype('int32')


# count proportion of each compound of how many appearances are sarcastic and how many are not
pro_sarcasm = pro_context.groupby('pattern')['quotes'].mean().to_dict()
con_sarcasm = con_context.groupby('pattern')['quotes'].mean().to_dict()

# map information for each compound to compounds data frame 
knowledge_base["pro_sarcasm"] = knowledge_base.original.map(pro_sarcasm)
knowledge_base["con_sarcasm"] = knowledge_base.original.map(con_sarcasm)

In [502]:
knowledge_base[knowledge_base.original == "klimafreund"].con_sarcasm

55    1.0
Name: con_sarcasm, dtype: float64

In [227]:
import spacy
from spacy import displacy
nlp = spacy.load('de_core_news_md')
from ast import literal_eval


In [12]:
def get_dependencies(df):
    
    """
    This function recursively retrieves the dependencies of words being dependent of a compound word. 
    Arg: 
        df: a data frame containing the compound words for which we want to count the POS tags of the dependent words.
    Returns: 
        A nested list consisting of the compound words as keys and the dependency information, 
        i.e. the dependent word, the POS tag of the dependent word, the dependency tag.
    """   
    
    #deps = dict.fromkeys(set(df.pattern)) # initiate a dictionary with the compounds as keys
    deps = []
    mods = ["mo", "mnr", "nk"] # the list of dependency tags we are interested in
    pos = ["ADJA", "ADJD", "PAV", "PROAV", "PDAT", "PIAT", "PIDAT",
          "PPOSAT", "PRELAT", "PTKA", "PWAT", "PWAV", "ADJ", "ADP", "ADV"] # the list of POS tags we are interested in
    
    # iterate over rows of data frame 
    for index, row in df.iterrows():
        
        doc = nlp(row["keyword"]) # retrieve sentence with keyword for according row
        
        tok_deps = [] # initiate empty list of dependencies for that row
        
        # for each token in the sentence 
        for token in doc:
        
            # check for words being dependent on our compound word, i.e. the compound is the head 
            if str(token.head.text).lower().startswith(row["pattern"].lower()):
                
                # if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                if token.dep_ in mods and token.tag_ in pos:
                        
                    # append information (lemma, token, pos tag, dependency tag, head word) to dependency list
                    tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])
                    
                    # use the new found word as new head 
                    new_head = token.text
                    
                    # check if we have words that are dependent on our new head
                    for token in doc:
                        
                        # if yes
                        if token.head.text == new_head:
                            
                            # and if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                            if token.dep_ in mods and token.tag_ in pos:
                                
                                # append it to our list
                                tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])


            # if we have a conjunct of one of the modifiers and another modifier
            if token.head.text == "und" and token.tag_ in pos and token.dep_ == "cj":
                
                # append to list
                tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])
                
                # use token as new head 
                next_head = token.text
                
                # check for dependent words 
                for token in doc:
                    
                    # if yes
                    if token.head.text == next_head: 
                        
                        # and if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                        if token.dep_ in mods and token.tag_ in pos:
                                
                            # append information to list
                            tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])

            # if none of the options apply, move on
            else:
                pass
                
        # append to final list 
        deps.append(tok_deps)
        
    # return final list with dependency information for each row in the data frame
    return deps 

In [16]:
def get_dependencies1(df):
    
    """
    This function recursively retrieves the dependencies of words being dependent of a compound word. 
    Arg: 
        df: a data frame containing the compound words for which we want to count the POS tags of the dependent words.
    Returns: 
        A nested list consisting of the compound words as keys and the dependency information, 
        i.e. the dependent word, the POS tag of the dependent word, the dependency tag.
    """   
    
    #deps = dict.fromkeys(set(df.pattern)) # initiate a dictionary with the compounds as keys
    deps = []
    mods = ["mo", "mnr", "nk"] # the list of dependency tags we are interested in
    pos = ["ADJA", "ADJD", "ADJ", "ADV"] # the list of POS tags we are interested in
    
    # iterate over rows of data frame 
    for index, row in df.iterrows():
        
        doc = nlp(row["keyword"]) # retrieve sentence with keyword for according row
        
        tok_deps = [] # initiate empty list of dependencies for that row
        
        # for each token in the sentence 
        for token in doc:
        
            # check for words being dependent on our compound word, i.e. the compound is the head 
            if str(token.head.text).lower().startswith(row["pattern"].lower()):
                
                # if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                if token.dep_ in mods and token.tag_ in pos:
                        
                    # append information (lemma, token, pos tag, dependency tag, head word) to dependency list
                    tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])
                    
                    # use the new found word as new head 
                    new_head = token.text
                    
                    # check if we have words that are dependent on our new head
                    for token in doc:
                        
                        # if yes
                        if token.head.text == new_head:
                            
                            # and if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                            if token.dep_ in mods and token.tag_ in pos:
                                
                                # append it to our list
                                tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])


            # if we have a conjunct of one of the modifiers and another modifier
            if token.head.text == "und" and token.tag_ in pos and token.dep_ == "cj":
                
                # append to list
                tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])
                
                # use token as new head 
                next_head = token.text
                
                # check for dependent words 
                for token in doc:
                    
                    # if yes
                    if token.head.text == next_head: 
                        
                        # and if the token is a modifier (i.e. token has one of the modifier tags and one of the POS tags that we defined above)
                        if token.dep_ in mods and token.tag_ in pos:
                                
                            # append information to list
                            tok_deps.append([token.lemma_, token.text, token.tag_, token.dep_, token.head.text])

            # if none of the options apply, move on
            else:
                pass
                
        # append to final list 
        deps.append(tok_deps)
        
    # return final list with dependency information for each row in the data frame
    return deps 

In [392]:
pro_context.reset_index(drop=True, inplace=True)

In [233]:
knowledge_base['compound_forms'] = knowledge_base.compound_forms.apply(lambda x: literal_eval(str(x)))
compound_forms = knowledge_base.compound_forms.tolist()
compound_forms = [item for sublist in compound_forms for item in sublist]

pro_context = pro_context[pro_context.keyword.str.contains(" |".join(compound_forms), case=False).groupby(level=0).any()]
con_context = con_context[con_context.keyword.str.contains(" |".join(compound_forms), case=False).groupby(level=0).any()]

In [247]:
compound_forms = knowledge_base[knowledge_base.original == "klimafreund"].compound_forms.tolist()
compound_forms

[['klimafreund',
  'klimafreundes',
  'klimafreunden',
  'klimafreunde',
  'klimafreunds']]

In [333]:
pro_context.iloc[19].keyword

'KlimaAktivist:innen ausgespäht ? !'

In [421]:
pro_context

,docname,from,to,pre,keyword,post,pattern
0,fff_de_00488.txt,5,5,Aber das macht nichts . Denn Du kannst trotzde...,Mit Deiner Spende unterstützt Du unseren Klima...,Jetzt spenden : Alle Zahlungen gehen ohne Abzu...,klimaaktivismus
1,farn236.txt,2,2,""" Auf Seiten beschreibt die Broschüre zunächst...","In einem weiteren Kapitel geht es darum , wie ...",Ein zentraler Teil der Broschüre ist ebenso di...,klimaaktivismus
2,farn260.txt,288,288,"„ Wenn es nach mir ginge , könnte die Klimaerw...",Auch seine Esoterik macht Storl anschlussfähig...,Die Simplizität der angebotenen Maßnahmen biet...,klimaaktivismus
3,fff_de_00130.txt,2,2,Why should I be studying for a future that soo...,And what is the point of learning facts when t...,"Wir sind eine Bewegung von jungen Menschen , d...",klimaaktivist
4,fff_de_00130.txt,12,12,Doch unsere Politiker * innen unternehmen nich...,Vorbild für unsere Aktion ist die KlimaAktivis...,Die 16jährige Schwedin bestreikt seit Monaten ...,klimaaktivist
...,...,...,...,...,...,...,...
798,fff_de_00417.txt,124,124,"„ Schlussrunde "" und alle haben sich wieder li...",Klimazerstörer wie RWE enteignen und verstaatl...,"Politiker die die Umweltzerstöng zulassen , en...",klimazerstörer
799,fff_de_00341.txt,2,2,.,RWE Hauptversammlung RWE ist der größte Klimaz...,"RWE hält an Kohlestrom fest , den niemand brau...",klimazerstörer
800,fff_de_00441.txt,163,163,"Prüft , welche Partei nur redet , aber nichts ...",Fragt sie welchen Einfluss sie nehmen können u...,Deutschland gehört zu den reichsten Ländern de...,klimazerstörer
801,fff_de_00250.txt,19,19,Ein Beitrag des #WaldStattAsphaltPlanungsteams...,"Geld für Klimaschutz … ausgeben , nicht für Kl...",) Auch die A44 ist eine Katastrophe für unsere...,klimazerstörung


In [438]:
#if pro_context.iloc[264].str.contains(" |".join(compound_forms), case = False).keyword:
 #   print("YAY")

In [437]:
#pro_context.iloc[124].keyword

In [436]:
#pro_context[pro_context.pattern == "klimagerechtigkeit"].keyword[262]

In [476]:
#for idx, row in pro_context.iterrows():
 #   print(idx)
" |".join(compound_forms)+" "

'klimazerstörung |klimazerstörungen '

In [479]:
def clean_context(df):

    # initiate list of rows that we have to drop
    to_drop = []
    
    # for each row in context data frame 
    for idx, row in df.iterrows():

        # retrieve compound word for according row 
        compound = row.pattern
        # retrieve compound forms for the compound word
        compound_forms = knowledge_base[knowledge_base.original == compound].compound_forms.tolist()
        # convert nested list into simplified list of compound forms 
        compound_forms = [item for sublist in compound_forms for item in sublist]

        # if the keyword phrase does not contain any of the compound forms 
        if not df.iloc[idx].str.contains(" |".join(compound_forms)+" ", case=False).keyword:
            print(compound)
            print(compound_forms)
            print(row.keyword)
            print("TO DROP")
            print("_"*20)
            # append this row index to the "to drop" list
            to_drop.append(idx)
            

        # if keyword phrase contains one of the compound forms
        else:
            # do nothing
            pass

    # drop rows that are contained in to_drop list
    df.drop(to_drop, inplace=True)
    
    # reset index of data frame 
    df.reset_index(drop=True, inplace=True)

In [480]:
clean_context(con_context)

klimaaktivist
['klimaaktivisten', 'klimaaktivist']
Anstatt mit einem Flugzeug in wenigen Stunden von Nordamerika nach Europa zurück zu fliegen , schiffte sich die Klimaaktivistin Greta Thunberg am .
TO DROP
____________________
klimaaktivist
['klimaaktivisten', 'klimaaktivist']
Gerrit Hansen von der Klimaaktivistengruppe Germanwatch .
TO DROP
____________________
klimaaktivist
['klimaaktivisten', 'klimaaktivist']
Dieses schreibt eine Klimaaktivistin aus Bangla Desh .
TO DROP
____________________
klimaaktivist
['klimaaktivisten', 'klimaaktivist']
Was ist die schwedische KlimaAktivistin Greta Thunberg denn nun : Heldin oder Nervensäge ?
TO DROP
____________________
klimaaktivist
['klimaaktivisten', 'klimaaktivist']
Seht , auch im schönen Isartal gibt es für das Klima besonders Engagierte Der ÖkoenergieAktivist durfte erzählen , wie grandios die Wasserstofftechnologie - woran er gerade arbeitet - unsere Energiewende retten wird , und die Freitagshüpferin ihr Wissen zum Klima : Klimaaktivi

In [481]:
for element in con_context[con_context.pattern == "klimafreund"].keyword:
    print(element)
    print("_"*10)

Moderne Heuschrecken sind geschworene „ Klimafreunde " Die für Eisbären begeisterten Schüler scheinen sich gar nicht dafür zu interessieren , dass die allerübelsten Heuschrecken heute in der Wall Street oder den Bankenvierteln anderer Finanzmetropolen sitzen und sich diebisch freuen , dass Kinder weltweit für ihre Profite trommeln .
__________


In [434]:
pro_context.reset_index(drop=True, inplace=True)

In [435]:
pro_context

,docname,from,to,pre,keyword,post,pattern
0,fff_de_00488.txt,5,5,Aber das macht nichts . Denn Du kannst trotzde...,Mit Deiner Spende unterstützt Du unseren Klima...,Jetzt spenden : Alle Zahlungen gehen ohne Abzu...,klimaaktivismus
1,farn236.txt,2,2,""" Auf Seiten beschreibt die Broschüre zunächst...","In einem weiteren Kapitel geht es darum , wie ...",Ein zentraler Teil der Broschüre ist ebenso di...,klimaaktivismus
2,farn260.txt,288,288,"„ Wenn es nach mir ginge , könnte die Klimaerw...",Auch seine Esoterik macht Storl anschlussfähig...,Die Simplizität der angebotenen Maßnahmen biet...,klimaaktivismus
3,fff_de_00130.txt,2,2,Why should I be studying for a future that soo...,And what is the point of learning facts when t...,"Wir sind eine Bewegung von jungen Menschen , d...",klimaaktivist
4,fff_de_00130.txt,12,12,Doch unsere Politiker * innen unternehmen nich...,Vorbild für unsere Aktion ist die KlimaAktivis...,Die 16jährige Schwedin bestreikt seit Monaten ...,klimaaktivist
...,...,...,...,...,...,...,...
590,fff_de_00417.txt,124,124,"„ Schlussrunde "" und alle haben sich wieder li...",Klimazerstörer wie RWE enteignen und verstaatl...,"Politiker die die Umweltzerstöng zulassen , en...",klimazerstörer
591,fff_de_00341.txt,2,2,.,RWE Hauptversammlung RWE ist der größte Klimaz...,"RWE hält an Kohlestrom fest , den niemand brau...",klimazerstörer
592,fff_de_00441.txt,163,163,"Prüft , welche Partei nur redet , aber nichts ...",Fragt sie welchen Einfluss sie nehmen können u...,Deutschland gehört zu den reichsten Ländern de...,klimazerstörer
593,fff_de_00250.txt,19,19,Ein Beitrag des #WaldStattAsphaltPlanungsteams...,"Geld für Klimaschutz … ausgeben , nicht für Kl...",) Auch die A44 ist eine Katastrophe für unsere...,klimazerstörung


In [ ]:
compound_forms = 

In [268]:
" |".join(compound_forms) in "hier gibt es klimazerstörung"

False

In [ ]:
def clean_context(df):
    for idx, row in df.iterrows():
    
        # für jede reihe: abchecken of keyword col contains one of the compound forms for the compound, append to new data frame?
        # retrieve compound word
        compound = row["pattern"]
        
        # retrieve compound forms for the according compound
        compound_forms = knowledge_base[knowledge_base.original == compound].compound_forms.tolist()
        
        # if row.keyword contains one of compound forms, append to new dataframe
                     
        df[df.keyword.str.contains(" |".join(compound_forms), case=False).groupby(level=0).any()]
    
    
    
    
    #string = row['keyword'].lower() # retrieve key word phrase
    #compound = row['pattern'] # retrieve compound word
    #print("COMPOUND:", compound)
    
    #print("STRING:", string)
       
    # retrieve exact positions of compound word in the string
    #for match in re.finditer(compound, string):
     #   start = match.start() # get start position
      #  end = match.end() # and end position
          

In [ ]:
for element in con_context[con_context.pattern == "klimafreund"].keyword:
    print(element)
    print("___________")
    

In [234]:
len(con_context)

1649

In [13]:
con_deps = get_dependencies(con_context)
pro_deps = get_dependencies(pro_context)

In [17]:
con_deps1 = get_dependencies1(con_context)
pro_deps1 = get_dependencies1(pro_context)

In [43]:
def get_mods(column):
    
    """
    This function retrieves the modifiers of the column created via the "get_dependencies" above. 
    Arg: 
        column: a data frame column containing list of dependencies from which we want to retrieve the very first element.
    Returns: 
        A list of modifier words for each compound word if possible, else 0.
    """ 
    
    mods = [] # initiate empty list
    
    # for each list of dependencies
    for deps in column:
        
        # if list ist empty
        if column == "[]":
            return 0 # return 0
           
        else:
            # if we have the string "innen" do not append to modifiers
            if deps[0] == "innen":
                pass
            
            # else 
            else:
            # get first element and save to new list
                mods.append(deps[0])
            
    return mods # return list of modifiers

In [44]:
# apply function to data frames 
con_deps = get_dependencies1(con_context)
pro_deps = get_dependencies1(pro_context)

# retrieve dependencies and save to new column in our data frame 
pro_context["dependencies"] = pro_deps
con_context["dependencies"] = con_deps

In [45]:
# apply modifier function to find all modifiers and save to new column "modifiers"
pro_context["modifiers"] = pro_context.dependencies.apply(get_mods)
con_context["modifiers"] = con_context.dependencies.apply(get_mods)

In [48]:
def flatten_list(nested_list):
    
    """
    This function flattens a nested list. 
    Arg: 
        nested_list: a nested list of the format [[item1],[item2]].
    Returns: 
        A flattened version of the nested list, e.g. [item1,item2], else NaN.
     """       
    
    try:
        return [item for sublist in nested_list for item in sublist]
    except:
        return np.nan

In [49]:
# convert modifiers column into dictionary
pro_mods = pro_context.groupby("pattern")["modifiers"].apply(list).to_dict()
con_mods = con_context.groupby("pattern")["modifiers"].apply(list).to_dict()

# map to compounds data frame
knowledge_base["pro_mods"] = knowledge_base.original.map(pro_mods)
knowledge_base["pro_mods"] = knowledge_base.pro_mods.apply(flatten_list)
knowledge_base["con_mods"] = knowledge_base.original.map(con_mods)
knowledge_base["con_mods"] = knowledge_base.con_mods.apply(flatten_list)

In [50]:
# get cleaned list of modifiers
mods_cleaned = pd.read_csv("../evaluation/modifiers_cleaned.csv", index_col = 0, sep=";")

# update knowledge base with cleaned list 
knowledge_base = knowledge_base.set_index('original')
knowledge_base.update(mods_cleaned)
knowledge_base.reset_index(inplace=True)

In [52]:
knowledge_base.to_csv("../evaluation/newKB.csv")

In [21]:
df_original = pd.DataFrame(con_deps)
df_new = pd.DataFrame(con_deps1)

In [27]:
pd.DataFrame(pro_deps).to_csv("../evaluation/deps_test.csv",index=False)
pd.DataFrame(pro_deps1).to_csv("../evaluation/deps_test_new.csv",index=False)

In [24]:
len(con_deps1)

1946

In [32]:
ne = (con_deps == con_deps1)

In [35]:
import numpy as np

In [41]:
ne_stacked = (df_original != df_new).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
#changed


difference_locations = np.where(df_original != df_new)
changed_from = df_original.values[difference_locations]
changed_to = df_new.values[difference_locations]
x = pd.DataFrame({'from': changed_from, 'to': changed_to}, index=changed.index)

In [42]:
# = [','.join(map(str, l)) for l in df['lists']]
x.to_csv("../evaluation/changes_deps.csv")

In [179]:
import re

In [235]:
def find_quotations(df):
    
    """
    This function checks for each key word sentence whether the compound words is mentioned in quotation marks. 
    Arg: 
        df: a data frame containing the compound and a key word phrase.
    Returns: 
        A new column with the binary label 1 if the compound is used in quotation marks, else 0.
    """ 
    
    # for each row in the data frame
    for idx, row in df.iterrows():
        
        string = row['keyword'].lower() # retrieve key word phrase
        compound = row['pattern'] # retrieve compound word
        #print("COMPOUND:", compound)
    
        #print("STRING:", string)
       
        # retrieve exact positions of compound word in the string
        for match in re.finditer(compound, string):
            start = match.start() # get start position
            end = match.end() # and end position
            
            #print(start, end)

            try: 
                # if 1 position before the starting point, we have one of the following quotation marks
                if string[start-1] == '"' or string[start-1] == """'""" or string[start-1] == """„""":
                    x = True # set x to True
                    
                    #print("!!!!FOUND1a!!!")

                # or if 2 position before the starting point, we have one of the following quotation marks
                elif string[start-2] == '"' or str1[start-2] == """'""" or string[start-2] == """„""":
                    x = True # set x to True
                    
                    #print("!!!FOUND1b!!!")
                    
                # else, x is False
                else:
                    x = False
                    
                    #print("!!!NOT_FOUND1!!!!")

            except:
                x = False
                y = False

            try:
                # if 1 position after the end point, we have one of the following quotation marks
                if string[end+1] == '"' or string[end+1] == """'""" or string[end+1] == """“""":
                    y = True # set y to True
                    
                    #print("!!!!FOUND2a!!!")
                    
                # if 2 positions after the end point, we have one of the following quotation marks
                elif string[end+2] == '"' or string[end+2] == """'""" or string[end+2] == """“""":
                    y = True # set y to True
                    
                    #print("!!!!FOUND2b!!!")
                    
                # else, y is False
                else:
                    y = False 
                    
                    #print("!!!!NOT_FOUND_2!!!")
                    
            except:
                x = False
                y = False

            # if we have 2 quotations marks surrounding the compound
            if x and y == True:
                df.at[idx,'quotes'] = 1 # set value in new column to 1 
                print("COMPOUND:", compound)
                print("SARCASM HERE")
 
            # if not:
            else:
                df.at[idx,'quotes'] = 0 # set value in new column to 0
                #print("NO SARCASM HERE")

In [236]:
# apply function to both data frames 
find_quotations(pro_context);
find_quotations(con_context);

COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimahysterie
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakanzlerin
SARCASM HERE
COMPOUND: klimakonsens
SARCASM HERE
COMPOUND: klimaleugner
SARCASM HERE
COMPOUND: klimalüge
SARCASM HERE
COMPOUND: klimapäckchen
SARCASM HERE
COMPOUND: klimaskeptiker
SARCASM HERE
COMPOUND: klimaskeptiker
SARCASM HERE
COMPOUND: klimaskeptiker
SARCASM HERE
COMPOUND: klimaskeptizismus
SARCASM HERE
COMPOUND: klimaaktivist
SARCASM HERE
COMPOUND: klimabibel
SARCASM HERE
COMPOUND: klimabremser
SARCASM HERE
COMPOUND: klimabremser
SARCASM HERE
COMPOUND: klimadiplomatie
SARCASM HERE
COMPOUND: klimaelite
SARCASM HERE


In [237]:
pro_context[pro_context.pattern == "klimahysterie"].quotes

594    1.0
595    1.0
596    0.0
597    0.0
598    1.0
599    1.0
600    1.0
601    1.0
602    1.0
603    0.0
Name: quotes, dtype: float64

In [458]:
for element in con_context[con_context.pattern == "klimafreund"].keyword:
    print(element)
    print("___________")

Moderne Heuschrecken sind geschworene „ Klimafreunde " Die für Eisbären begeisterten Schüler scheinen sich gar nicht dafür zu interessieren , dass die allerübelsten Heuschrecken heute in der Wall Street oder den Bankenvierteln anderer Finanzmetropolen sitzen und sich diebisch freuen , dass Kinder weltweit für ihre Profite trommeln .
___________
Ich kann aber für den kommenden Winter versichern , dass niemand in seiner Wohnung frieren wird infolge Mangels an Kohle " . - Faule Ausreden : KlimaFreundschaftspakt zwischen China und Europa kommt ins Schwimmen Bonn - Es war vorgesehen , dass die EU und China zu einer Arbeitsgemeinschaft internationaler KlimaDiplomatie werden sollten .
___________


In [214]:
#str1 = """letzteres hält man für „ panikmache " und „ klimahysterie "."""
str1 = """also kein grund zur entwarnung , ganz im gegenteil , so rast der zug mit hoher wahrscheinlichkeit in den abgrund … ( s . dazu meine kleine parabel weiter oben ) wer was von mir zur „ klimahysterie " lesen will , zu verrückten omas aufm motorrad und ähnlich gelagerten intendanten ( vom ."""

print(str1)

for match in re.finditer("klimahysterie", str1):
    start = match.start() # get start position
    end = match.end() # and end position


if str1[start-1] == '"' or str1[start-1] == """'""" or str1[start-1] == """„""":
    print(True)
elif str1[start-2] == '"' or str1[start-2] == """'""" or str1[start-2] == """„""":
    print(True)


else:
    print(False)

also kein grund zur entwarnung , ganz im gegenteil , so rast der zug mit hoher wahrscheinlichkeit in den abgrund … ( s . dazu meine kleine parabel weiter oben ) wer was von mir zur „ klimahysterie " lesen will , zu verrückten omas aufm motorrad und ähnlich gelagerten intendanten ( vom .
True


In [215]:
pro_context[pro_context.quotes == 1.0]

,docname,from,to,pre,keyword,post,pattern,quotes
594,fff_de_00154.txt,110,110,Klimafolgenverharmloser Das ist der neuste Typ...,"Letzteres hält man für „ PanikMache "" und „ Kl...",Den entsprechenden KlimaschutzBewegungen unter...,klimahysterie,1.0
595,fff_de_00154.txt,217,217,Als nächstes betrachten wir mal die Zahlen von...,"Also kein Grund zur Entwarnung , ganz im Gegen...",) Hoffe das war nicht zu kompliziert und von I...,klimahysterie,1.0
598,fff_de_00037.txt,3,3,Was für eine turbulente Woche vom . -19.1 . - ...,"Wie schon im Titel zu lesen , bekam das Wort „...","Die Entscheidung wird damit begründet , dass d...",klimahysterie,1.0
599,fff_de_00199.txt,170,170,Dieser Wissensaustausch erhöht das Veränderung...,eine kleine Fallstudie ( und unten was zur „ K...,"Blätterwald , die TVMagazine ZAP ( NDR ) , Kul...",klimahysterie,1.0
600,fff_de_00199.txt,174,174,Mehr Infos sind auf dieser Webseite zu finden ...,Also Klimaleugner wenden sich gegen eine Satir...,Der Anlass des Ganzen : Ein satirischer VideoB...,klimahysterie,1.0
601,fff_de_00199.txt,212,212,Aber alles was medial in die Richtung einer ku...,"Aktuell wurde „ Klimahysterie "" zum Unwort des...","Dies finde ich ein richtiges Signal , denn wie...",klimahysterie,1.0
602,farn168.txt,30,30,Meist wird nur ein Teil von wissenschaftlichen...,Denn es gäbe dringlichere globale Probleme als...,Außerdem koste das alles viel Geld . Bei diese...,klimahysterie,1.0
621,fff_de_00019.txt,822,822,"Alter macht nicht immer schlau , bzw . zerstör...",Ein Wahlkampf wo dies Thema faktisch unbedeute...,) in Wahlarenen das Blaue vom Himmel herunter ...,klimakanzlerin,1.0
622,fff_de_00195.txt,267,267,"Die USA müsste das sogar bis schaffen , wenn a...","Also ran an die politisch Verantwortlichen , v...",Der hartleibigste Sektor ist der Verkehr . Sch...,klimakanzlerin,1.0
623,fff_de_00199.txt,28,28,Der Deutsche Wetterdienst warnt vor den Folgen...,Die beschönigte Rede der ehemaligen „ Klimakan...,Debatte ums Tempolimit : Guter Menschenverstan...,klimakanzlerin,1.0


In [216]:
con_context[con_context.quotes == 1.0]

,docname,from,to,pre,keyword,post,pattern,quotes
44,eike_07528.txt,312,312,"Diese werden deutlicher , wenn Sie nicht die W...","„ Klimaaktivist "" die Wahrheit über unser „ no...","Aber , dies muss Ihnen ja auch nicht gefallen ...",klimaaktivist,1.0
507,eike_01129.txt,24,24,"John Ioannidis , der im Zusammenhang mit COVID...","Die viel gepriesene „ Klimabibel "" des IPCC ( ...","Eine gängige Propagandatechnik besteht darin ,...",klimabibel,1.0
515,eike_01053.txt,131,131,Beim EAuto dagegen wird man politisch und fina...,"Wobei das Wort „ Klimabremser "" nicht mal in A...",Wollen sie mal bitte mit ihren eigenen Worten ...,klimabremser,1.0
516,eike_01053.txt,132,132,Das Einzige was dagegen hilft ist auf Auto kom...,Wollen sie mal bitte mit ihren eigenen Worten ...,"Sind das alle die Menschen , die wollen , dass...",klimabremser,1.0
542,eike_05122.txt,12,12,"a ) X GW Kernkraft , b ) X MW Windturbinen , c...","Frage : Falls keine „ KlimaDiplomatie "" gegebe...","Gibt es einen Zeitraum von Jahren , den man ve...",klimadiplomatie,1.0
...,...,...,...,...,...,...,...,...
1811,eike_05094.txt,144,144,"„ TopTenRanking "" zu erstellen . Aber mir ist ...",Nun sind es Zitate geworden … Ich hätte es eig...,"Und ich „ befürchte "" , ich könnte bis zum Kon...",klimatrash,1.0
1833,eike_08554.txt,37,37,Ebenfalls aus Deutschland wartete die Financia...,Im Artikel der Financial Times wird die Greenp...,Inzwischen werden die Besatzungsmitglieder ein...,klimaverbrechen,1.0
1845,eike_05960.txt,43,43,Natürlich alles nur im Rahmen des Kampfes gege...,Lassen wir nichts liegen War eigentlich der Au...,"Konnte mir nicht vorstellen , daß solch eine g...",klimaverteidigung,1.0
1923,eike_02431.txt,140,140,"Ihre „ Wissenschaft "" schafft Umsatz für besti...",Dabei schreckt man vor nichts zurück - rotgrün...,"Ihr logischer Schluss : Nicht ich , sondern di...",klimaweltuntergang,1.0


In [217]:
con_context[con_context.pattern == "klimafreund"]

,docname,from,to,pre,keyword,post,pattern,quotes
582,eike_10957.txt,113,113,"Je mehr Stromtrassen , desto mehr Kohlestrom ,...",Warum sollte man den Ausbau günstiger und klim...,Wir könnten bis den Anteil erneuerbarer Energi...,klimafreund,0.0
583,eike_10744.txt,16,16,Sie haben auch noch keinerlei Erfahrungen dami...,Moderne Heuschrecken sind geschworene „ Klimaf...,"Denn „ Klimaschutz "" ist inzwischen das Jagdre...",klimafreund,1.0
584,eike_10213.txt,66,66,Der Reaktortyp hat überhaupt nichts mit den in...,"Die Grünen sorgten sich , es könne sich die in...","Sie würden in erschreckender Weise fordern , d...",klimafreund,0.0
585,eike_11563.txt,67,67,Wow ! Deine aufgestachelten Worte entspringen ...,"Liebe Greta , wir lebten damals klimafreundlic...",Wir trugen die Kleider unserer älteren Geschwi...,klimafreund,0.0
586,eike_11563.txt,85,85,Jugend ist kein Vorbild Wie scheinheilig Eure ...,Gerade Ihr seid deshalb kein Vorbild für eine ...,"Deshalb , liebe Greta , nimm lieber Deine Alte...",klimafreund,0.0
587,eike_01300.txt,36,36,Dies ist eine der zahlreichen Maßnahmen von Re...,"Die Bundesregierung pocht zudem darauf , Gelde...","„ Das hat nichts mit dem zu tun , was wir uns ...",klimafreund,0.0
588,eike_03529.txt,8,8,"Schließlich gilt es , bedingungs- und alternat...",… Der Wandel wird die Mobilität zukünftig nich...,Eine erfolgreiche technologische Transformatio...,klimafreund,0.0
589,eike_03529.txt,10,10,Und ein wichtiger Teil dieser Strategie ist da...,So kann Deutschland seine Stellung als innovat...,Mit diesen Technologien kann die Industrie ein...,klimafreund,0.0
590,eike_03529.txt,27,27,"Und deshalb macht unsere Politik alles , um au...",„ Dass allein drei der bundesweit sieben Lehrs...,Die Stiftungsprofessuren sind Teil des nationa...,klimafreund,0.0
591,eike_03529.txt,40,40,"Viel moderner , sicher Gentechnikfrei und mind...",Die heute für die Zerschlagung der konventione...,Dieses Bundesland will Vorreiter sein . Dabei ...,klimafreund,0.0


In [203]:
con_context[con_context.pattern == "klimafreund"].quotes.values

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [194]:
# convert type of number from float to integer
pro_context.quotes = pro_context.quotes.astype('int32')
con_context.quotes = con_context.quotes.astype('int32')

In [199]:
pro_context[pro_context.quotes != 0]

pro_sarcasm = pro_context.groupby('pattern')['quotes'].mean().to_dict()
con_sarcasm = con_context.groupby('pattern')['quotes'].mean().to_dict()



In [201]:
con_sarcasm

{'klimaabzockerei': 0.0,
 'klimaaktivismus': 0.0,
 'klimaaktivist': 0.010416666666666666,
 'klimaaktivistin': 0.0,
 'klimaalarm': 0.0,
 'klimaalarmist': 0.0,
 'klimaanbeter': 0.0,
 'klimaapokalypse': 0.0,
 'klimaapokalyptiker': 0.0,
 'klimaapostel': 0.0,
 'klimaargument': 0.0,
 'klimaarmageddon': 0.0,
 'klimaaufstand': 0.0,
 'klimabankrott': 0.0,
 'klimabeeinflussung': 0.0,
 'klimabefürworter': 0.0,
 'klimabesoffenheit': 0.0,
 'klimabesorgnis': 0.0,
 'klimabetrug': 0.0,
 'klimabetrüger': 0.0,
 'klimabewegtheit': 0.0,
 'klimabibel': 1.0,
 'klimabigotterie': 0.0,
 'klimabluff': 0.0,
 'klimablödsinn': 0.0,
 'klimabrandstifter': 0.0,
 'klimabremser': 1.0,
 'klimachaos': 0.0,
 'klimacrash': 0.0,
 'klimadampfer': 0.0,
 'klimademagoge': 0.0,
 'klimadepression': 0.0,
 'klimadialektik': 0.0,
 'klimadiktatur': 0.0,
 'klimadiplomatie': 0.16666666666666666,
 'klimadogma': 0.0,
 'klimadonna': 0.0,
 'klimadramatik': 0.0,
 'klimadürre': 0.0,
 'klimaelite': 0.2,
 'klimaerkrankung': 1.0,
 'klimaerzählu

In [ ]:
pro_context.quotes = pro_context.quotes.astype('int32')
con_context.quotes = con_context.quotes.astype('int32')


# count proportion of each compound of how many appearances are sarcastic and how many are not
pro_sarcasm = pro_context.groupby('pattern')['quotes'].mean().to_dict()
con_sarcasm = con_context.groupby('pattern')['quotes'].mean().to_dict()

# map information for each compound to compounds data frame 
compounds["pro_sarcasm"] = compounds.original.map(pro_sarcasm)
compounds["con_sarcasm"] = compounds.original.map(con_sarcasm)

In [88]:
# function to get full text (combination of pre, keyword and post column) for each keyword, i.e. compound word 

def get_full_text(df):
    
    """
    Retrieves the full text preceding and following the keyword phrase and saves full text to new column.
    Arg: 
        df: the data frame containing the keyword-in-context information (columns "pre", "keyword" and "post").
    Returns: 
        The original data frame with a new column "full" containing the combined text columns. 
    """
    
    df = df.replace(np.nan,'',regex=True)
    
    # create full text for each column
    df["full"] = df["pre"] + df["keyword"] + df["post"]
    
    # convert full text column to string
    df.full = df.full.astype(str)
    
    return df  

In [89]:
# apply function to both data frames 
pro_context = get_full_text(pro_context)
con_context = get_full_text(con_context)

In [90]:
pro_context

,docname,from,to,pre,keyword,post,pattern,quotes,full
0,fff_de_00488.txt,5,5,Aber das macht nichts . Denn Du kannst trotzde...,Mit Deiner Spende unterstützt Du unseren Klima...,Jetzt spenden : Alle Zahlungen gehen ohne Abzu...,klimaaktivismus,0.0,Aber das macht nichts . Denn Du kannst trotzde...
1,farn236.txt,2,2,""" Auf Seiten beschreibt die Broschüre zunächst...","In einem weiteren Kapitel geht es darum , wie ...",Ein zentraler Teil der Broschüre ist ebenso di...,klimaaktivismus,0.0,""" Auf Seiten beschreibt die Broschüre zunächst..."
2,farn260.txt,288,288,"„ Wenn es nach mir ginge , könnte die Klimaerw...",Auch seine Esoterik macht Storl anschlussfähig...,Die Simplizität der angebotenen Maßnahmen biet...,klimaaktivismus,0.0,"„ Wenn es nach mir ginge , könnte die Klimaerw..."
3,fff_de_00130.txt,2,2,Why should I be studying for a future that soo...,And what is the point of learning facts when t...,"Wir sind eine Bewegung von jungen Menschen , d...",klimaaktivist,0.0,Why should I be studying for a future that soo...
4,fff_de_00130.txt,12,12,Doch unsere Politiker * innen unternehmen nich...,Vorbild für unsere Aktion ist die KlimaAktivis...,Die 16jährige Schwedin bestreikt seit Monaten ...,klimaaktivist,0.0,Doch unsere Politiker * innen unternehmen nich...
...,...,...,...,...,...,...,...,...,...
798,fff_de_00417.txt,124,124,"„ Schlussrunde "" und alle haben sich wieder li...",Klimazerstörer wie RWE enteignen und verstaatl...,"Politiker die die Umweltzerstöng zulassen , en...",klimazerstörer,0.0,"„ Schlussrunde "" und alle haben sich wieder li..."
799,fff_de_00341.txt,2,2,.,RWE Hauptversammlung RWE ist der größte Klimaz...,"RWE hält an Kohlestrom fest , den niemand brau...",klimazerstörer,0.0,.RWE Hauptversammlung RWE ist der größte Klima...
800,fff_de_00441.txt,163,163,"Prüft , welche Partei nur redet , aber nichts ...",Fragt sie welchen Einfluss sie nehmen können u...,Deutschland gehört zu den reichsten Ländern de...,klimazerstörer,0.0,"Prüft , welche Partei nur redet , aber nichts ..."
801,fff_de_00250.txt,19,19,Ein Beitrag des #WaldStattAsphaltPlanungsteams...,"Geld für Klimaschutz … ausgeben , nicht für Kl...",) Auch die A44 ist eine Katastrophe für unsere...,klimazerstörung,0.0,Ein Beitrag des #WaldStattAsphaltPlanungsteams...
